<a href="https://colab.research.google.com/github/jayashree-codes/Applied_Machine_Learning_Species/blob/main/Spatial_Extrapolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#SPATIAL EXTRAPOLATION FOR TEMPERATURE AND PRECIPITATION - TRAIN DATA
import pandas as pd
import numpy as np

excel_path = "/Users/jayashreehariharan/Desktop/AML_Project/Species-2/species/Merged_train_data.xlsx"
sheet_name = "merged_data"
df = pd.read_excel(excel_path, sheet_name=sheet_name)

cols_to_fill = [
    'tavg_2020_mean','prec_2020_mean',
    'tavg_2021_mean','prec_2021_mean',
    'tavg_2022_mean','prec_2022_mean',
    'tavg_2023_mean','prec_2023_mean',
    'tavg_2024_mean','prec_2024_mean'
]

valid = df.dropna(subset=cols_to_fill, how='all').reset_index(drop=True)
missing = df[df[cols_to_fill].isna().any(axis=1)].reset_index()

def haversine_vectorized(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1[:, None]
    dlon = lon2 - lon1[:, None]
    a = np.sin(dlat/2)**2 + np.cos(lat1)[:, None] * np.cos(lat2) * np.sin(dlon/2)**2
    a = np.clip(a, 0, 1)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

dist_matrix = haversine_vectorized(
    missing['latitude'].values,
    missing['longitude'].values,
    valid['latitude'].values,
    valid['longitude'].values
)

nearest_indices = np.argmin(dist_matrix, axis=1)

for idx_missing, idx_valid in zip(missing['index'], nearest_indices):
    for col in cols_to_fill:
        if pd.isna(df.at[idx_missing, col]):
            df.at[idx_missing, col] = valid.at[idx_valid, col]

with pd.ExcelWriter(excel_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name=sheet_name, index=False)

In [2]:
#SPATIAL EXTRAPOLATION FOR TEMPERATURE AND PRECIPITATION - TEST DATA
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

excel_path = "/Users/jayashreehariharan/Desktop/AML_Project/Species-2/species/species_test.xlsx"
sheet_name = "test_locs"
df = pd.read_excel(excel_path, sheet_name=sheet_name)

cols_to_fill = [
    'tavg_2020_mean','prec_2020_mean',
    'tavg_2021_mean','prec_2021_mean',
    'tavg_2022_mean','prec_2022_mean',
    'tavg_2023_mean','prec_2023_mean',
    'tavg_2024_mean','prec_2024_mean'
]

valid = df.dropna(subset=cols_to_fill, how='all').reset_index(drop=True)
missing = df[df[cols_to_fill].isna().any(axis=1)].reset_index()

def latlon_to_xyz(lat, lon):
    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)
    x = np.cos(lat_rad) * np.cos(lon_rad)
    y = np.cos(lat_rad) * np.sin(lon_rad)
    z = np.sin(lat_rad)
    return np.column_stack((x, y, z))

valid_xyz = latlon_to_xyz(valid['latitude'].values, valid['longitude'].values)
missing_xyz = latlon_to_xyz(missing['latitude'].values, missing['longitude'].values)

tree = cKDTree(valid_xyz)

_, nearest_idx = tree.query(missing_xyz, k=1)

for idx_missing, idx_valid in zip(missing['index'], nearest_idx):
    for col in cols_to_fill:
        if pd.isna(df.at[idx_missing, col]):
            df.at[idx_missing, col] = valid.at[idx_valid, col]

with pd.ExcelWriter(excel_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Missing values filled")

Missing values filled successfully!
